## Required imports and insallations

In [ ]:
from IPython.display import clear_output
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import time


!pip install --quiet torch joblib
clear_output()

## Data processing

In [ ]:
hol = pd.read_pickle('/content/holidays_processed.pkl')
fl = pd.read_pickle('/content/flows_processed.pkl')
sp = pd.read_pickle('/content/speeds_processed.pkl')
wea = pd.read_pickle('/content/weather_processed.pkl')
temp = pd.read_pickle('/content/temporal_processed.pkl')
f_savg = pd.read_pickle('/content/flows_SAVG_processed.pkl')
s_savg = pd.read_pickle('/content/speeds_SAVG_processed.pkl')
w_savg = pd.read_pickle('/content/weather_SAVG_processed.pkl')
f_mavg = pd.read_pickle('/content/flows_MAVG_processed.pkl')
s_mavg = pd.read_pickle('/content/speeds_MAVG_processed.pkl')
w_mavg = pd.read_pickle('/content/weather_MAVG_processed.pkl')

In [ ]:
dataframes = [hol, fl, sp, wea, temp, f_savg, s_savg, w_savg, f_mavg, s_mavg, w_mavg]
for df in dataframes:
    df.columns = df.columns.astype(str)

In [ ]:
num_nodes = fl.shape[1]
print('Number of nodes: ', num_nodes)

In [ ]:
hd= temp['hour_of_day'].copy().to_frame()
su = hol['is_sunday'].copy().to_frame()
t = wea['temp'].copy().to_frame()
t_savg = w_savg['temp'].copy().to_frame()
t_mavg = w_mavg['temp'].copy().to_frame()
ws = wea['windspeed'].copy().to_frame()
ws_savg = w_savg['windspeed'].copy().to_frame()
ws_mavg = w_mavg['windspeed'].copy().to_frame()
h = wea['humidity'].copy().to_frame()
h_savg = w_savg['humidity'].copy().to_frame()
h_mavg = w_mavg['humidity'].copy().to_frame()

In [ ]:
general_data_s1 =  pd.concat([ f_mavg,  f_savg, sp, s_mavg, s_savg, wea,  w_mavg, w_savg,  temp, hol ], axis = 1)
general_data_s2 =  pd.concat([ f_mavg,  f_savg, sp, s_mavg, s_savg,                        temp, hol ], axis = 1)
general_data_s3 =  pd.concat([ f_mavg,  f_savg, sp, s_mavg, s_savg, wea,  w_mavg, w_savg,  temp      ], axis = 1)
general_data_s4 =  pd.concat([                                      wea,  w_mavg, w_savg,  temp, hol ], axis = 1)
general_data_s5 =  pd.concat([                  sp,                 wea,                   temp, hol ], axis = 1)
general_data_s6 =  pd.concat([                  sp,                                        temp, hol ], axis = 1)
general_data_s7 =  pd.concat([                  sp,                 wea,                   temp      ], axis = 1)
general_data_s8 =  pd.concat([                                      wea,                   temp, hol ], axis = 1)
general_data_s9 =  pd.concat([ f_mavg,  f_savg, sp, s_mavg, s_savg, t, t_savg, t_mavg, ws, ws_savg, ws_mavg, h, h_savg, h_mavg, hd, su], axis = 1)
general_data_s10 = pd.concat([ sp,                                  t,                 ws,                   h,                 hd, su], axis = 1)

general_data_list = [general_data_s1, general_data_s2, general_data_s3, general_data_s4, general_data_s5,
                     general_data_s6, general_data_s7, general_data_s8, general_data_s9, general_data_s10]

for i in range(len(general_data_list)):
  print('General data ', i+1, ' shape: ', general_data_list[i].shape)

In [ ]:
flow_scaler = MinMaxScaler()
speed_scaler = MinMaxScaler()
general_scaler = MinMaxScaler()

flow_data_scaled = pd.DataFrame(flow_scaler.fit_transform(fl).astype(float), columns=fl.columns)


general_data_scaled_list = []

# Iterate over the dataframes and apply scaling
for data in general_data_list:
    scaled_data = pd.DataFrame(general_scaler.fit_transform(data).astype(float), columns=data.columns)
    general_data_scaled_list.append(scaled_data)

joblib.dump(flow_scaler, 'flow_scaler.gz')
# joblib.dump(speed_scaler, 'speed_scaler.gz')
# joblib.dump(general_scaler, 'general_scaler.gz')

full_data_list = []
for i in range(len(general_data_scaled_list)):
  if i == 3 or i == 7:
    full_data = general_data_scaled_list[i].copy()
    full_data_list.append(full_data)
  else:
    full_data = pd.concat([flow_data_scaled, general_data_scaled_list[i]], axis = 1)
    full_data_list.append(full_data)
  print('Full data ', i+1, ' shape: ', full_data.shape)



Data split

In [ ]:
pred_hor = 12
scenario = 1
full_data = full_data_list[scenario-1]

In [ ]:
days_for_training = int(334*0.7)
days_for_testing = int(334*0.2) + 1
days_for_validation = int(334*0.1)

test_steps = days_for_testing * 24
validation_steps = days_for_validation * 24
training_steps = days_for_training * 24

training_data = full_data[:training_steps]
validation_data = full_data[training_steps : training_steps + validation_steps]
test_data = full_data[training_steps + validation_steps :]

# training_data = full_data[:training_steps]
# validation_data = full_data[training_steps :]

print("Training data shape: ", training_data.shape)
print("Validation data shape: ", validation_data.shape)
print("Test data shape: ", test_data.shape)


In [ ]:
# Define the sequence creation function
def create_nodata_sequences(inputs, labels, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(inputs) - n_steps_in - n_steps_out + 1):
        seq_x = inputs.iloc[i:i + n_steps_in].values
        seq_y = labels.iloc[i + n_steps_in:i + n_steps_in + n_steps_out].values
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
def create_sequences(data, n_steps_in, n_steps_out, num_features):
    if n_steps_out > n_steps_in:
        print("Warning: n_steps_out is greater than n_steps_in. Make sure this is intended.")

    if num_features > data.shape[1]:
        raise ValueError("num_features exceeds the number of features in the data.")

    X, y = [], []
    for i in range(len(data) - n_steps_in - n_steps_out + 1):  # Corrected loop range
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        seq_x, seq_y = data[i:end_ix, :], data[end_ix:out_end_ix, :num_features]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)



In [ ]:
training_data_values = training_data.values
test_data_values = test_data.values
validation_data_values = validation_data.values

n_features = full_data.shape[1]
n_steps_in, n_steps_out = 12, pred_hor
X_train, y_train = create_sequences(training_data_values, n_steps_in, n_steps_out, num_nodes)
X_validation, y_validation = create_sequences(validation_data_values, n_steps_in, n_steps_out, num_nodes)
X_test, y_test = create_sequences(test_data_values, n_steps_in, n_steps_out, num_nodes)

print('X_train length: ', len(X_train), ' and shape ', X_train[1].shape)
print('Y_train length: ', len(y_train), ' and shape ', y_train[1].shape)

print('X_validation length: ', len(X_validation), ' and shape ', X_validation[1].shape)
print('Y_validation length: ', len(y_validation), ' and shape ', y_validation[1].shape)

print('X_test length: ', len(X_test), ' and shape ', X_test[1].shape)
print('Y_test length: ', len(y_test), ' and shape ', y_test[1].shape)


Creating tensors


In [ ]:
X_train_tensor = torch.tensor(X_train).float()
y_train_tensor = torch.tensor(y_train).float()
X_test_tensor = torch.tensor(X_test).float()
y_test_tensor = torch.tensor(y_test).float()
X_validation_tensor = torch.tensor(X_validation).float()
y_validation_tensor = torch.tensor(y_validation).float()

train_data_final = TensorDataset(X_train_tensor, y_train_tensor)
test_data_final = TensorDataset(X_test_tensor, y_test_tensor)
validation_data_final = TensorDataset(X_validation_tensor, y_validation_tensor)

batch_size = 32
train_loader = DataLoader(train_data_final, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_data_final, shuffle=False, batch_size=batch_size)
validation_loader = DataLoader(validation_data_final, shuffle=False, batch_size=batch_size)

print(X_train_tensor.shape)
print(y_train_tensor.shape)
print(X_test_tensor.shape)
print(y_test_tensor.shape)
print(X_validation_tensor.shape)
print(y_validation_tensor.shape)

## Model Definition

### ATT-Bi-LSTM

In [ ]:
class AttentionLayer(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionLayer, self).__init__()
        self.hidden_dim = hidden_dim
        self.attention_weights = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.context_vector = nn.Linear(hidden_dim * 2, 1, bias=False)

    def forward(self, lstm_output):
        attention_scores = self.context_vector(torch.tanh(self.attention_weights(lstm_output)))
        attention_weights = torch.softmax(attention_scores, dim=1)
        context_vector = attention_weights * lstm_output
        context_vector = torch.sum(context_vector, dim=1)
        return context_vector

class ATT_BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, pred_horizon):
        super(ATT_BiLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.pred_horizon = pred_horizon

        # Bidirectional LSTM
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.attention = AttentionLayer(hidden_dim)
        self.fc = nn.Linear(hidden_dim * 2, output_dim * pred_horizon)
        self.activation = nn.ReLU()

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers * 2, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, batch_size, self.hidden_dim).to(x.device)

        # LSTM layer
        lstm_out, _ = self.lstm(x, (h0, c0))

        # Apply attention
        context_vector = self.attention(lstm_out)

        # Fully connected layer
        out = self.fc(context_vector)

        # Activation
        predicted_flows = self.activation(out)

        # Reshape to have the prediction horizon
        predicted_flows = predicted_flows.view(batch_size, self.pred_horizon, -1)
        return predicted_flows

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_features = X_train_tensor.shape[2]
output_features = y_train_tensor.shape[2]
pred_horizon = n_steps_out

# Initialize the model with the desired prediction horizon
model = ATT_BiLSTMModel(input_dim=input_features, hidden_dim=300, num_layers=3, output_dim=output_features, pred_horizon=pred_horizon)
model.to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Dropout model

In [ ]:
class AttentionLayer(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionLayer, self).__init__()
        self.hidden_dim = hidden_dim
        self.attention_weights = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.context_vector = nn.Linear(hidden_dim * 2, 1, bias=False)

    def forward(self, lstm_output):
        attention_scores = self.context_vector(torch.tanh(self.attention_weights(lstm_output)))
        attention_weights = torch.softmax(attention_scores, dim=1)
        context_vector = attention_weights * lstm_output
        context_vector = torch.sum(context_vector, dim=1)
        return context_vector

class ATT_BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, pred_horizon, dropout):
        super(ATT_BiLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.pred_horizon = pred_horizon

        # Bidirectional LSTM with dropout
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True, dropout=dropout)
        self.attention = AttentionLayer(hidden_dim)
        self.fc = nn.Linear(hidden_dim * 2, output_dim * pred_horizon)
        self.activation = nn.ReLU()

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers * 2, batch_size, self.hidden_dim).to(x.device)  # 2 for bidirection
        c0 = torch.zeros(self.num_layers * 2, batch_size, self.hidden_dim).to(x.device)

        # LSTM layer
        lstm_out, _ = self.lstm(x, (h0, c0))

        # Apply attention
        context_vector = self.attention(lstm_out)

        # Fully connected layer
        out = self.fc(context_vector)

        # Activation
        predicted_flows = self.activation(out)

        # Reshape to have the prediction horizon
        predicted_flows = predicted_flows.view(batch_size, self.pred_horizon, -1)
        return predicted_flows

In [ ]:
# Assuming X_train_tensor and y_train_tensor are your training data tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_features = X_train_tensor.shape[2]
output_features = y_train_tensor.shape[2]
pred_horizon = n_steps_out
dropout_rate = 0.2
# Initialize the model with the desired prediction horizon and dropout rate
model = ATT_BiLSTMModel(input_dim=input_features, hidden_dim=300, num_layers=3, output_dim=output_features, pred_horizon=pred_horizon, dropout=dropout_rate)
model.to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Loss functions definition

In [ ]:
def rmse(predictions, targets):
    return torch.sqrt(((predictions - targets) ** 2).mean())

def mae(predictions, targets):
    return torch.abs(predictions - targets).mean()

def mape_loss(outputs, labels):
    # Avoid division by zero; replace zero actuals with a small number (epsilon)
    epsilon = 1e-8
    return torch.mean(torch.abs(labels - outputs) / (labels + epsilon)) * 100

def mse_loss(output, label):
    # Ensure the output and label tensors are the same shape
    assert output.shape == label.shape, "Output and label must have the same shape"

    # Compute the mean squared error
    mse = torch.mean((output - label) ** 2)
    return mse

## Train and validate model

In [ ]:
import torch
import os
import time

# Define the training function with model checkpointing
def train_model(model, train_loader, validation_loader, criterion, optimizer, num_epochs, patience):
    best_val_loss = float('inf')
    patience_counter = 0
    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        total_train_loss = 0
        # Start timer

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)  # Use the specified criterion (MSE in your case)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        total_train_loss /= len(train_loader)
        # if (epoch + 1) % 20 == 0:
        #     print(f"Epoch {epoch+1} Train Loss: {total_train_loss:.6f}")

        # Validation
        model.eval()  # Set model to evaluation mode
        total_val_loss = 0
        with torch.no_grad():
            for inputs, labels in validation_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()

        total_val_loss /= len(validation_loader)
        # Only printing after every 20 epochs
        # if (epoch + 1) % 20 == 0:
        #     print(f"Epoch {epoch+1} Validation Loss: {total_val_loss:.6f}")

        # Checkpointing and Early Stopping
        if total_val_loss < best_val_loss:
            best_val_loss = total_val_loss
            patience_counter = 0
            # Save the model checkpoint
            torch.save(model.state_dict(), f'best_model.pth')
            # print(f"Epoch {epoch+1}: New best model saved with validation loss: {total_val_loss:.6f}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                # print(f"Early stopping at epoch {epoch+1} due to no improvement in validation loss for {patience} epochs.")
                break
        # End timer
        end_time = time.time()
        time_to_train = end_time - start_time

    return best_val_loss, time_to_train  # Return the best validation loss achieved


# best_val_loss, time_to_train = train_model(model, train_loader, validation_loader, criterion, optimizer, num_epochs=20, patience=20)
# print(f"Best validation loss: {best_val_loss:.6f}")
# print(f"Total training time: {time_to_train:.2f} seconds")


# model.load_state_dict(torch.load('/content/best_model.pth'))

In [ ]:
import torch
import numpy as np
import joblib
import pandas as pd  # Import pandas

epsilon = 1e-4

def evaluate_model_and_collect_data(model, test_loader, device, scaler_path, pred_horizon):
    model.eval()
    predictions = []
    actuals = []
    errors = {
        'mse': [[] for _ in range(pred_horizon)],
        'rmse': [[] for _ in range(pred_horizon)],
        'mae': [[] for _ in range(pred_horizon)],
        'mape': [[] for _ in range(pred_horizon)]
    }

    flow_scaler = joblib.load(scaler_path)

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            outputs_np = outputs.cpu().numpy()
            labels_np = labels.cpu().numpy()

            # Flatten the predictions and labels for inverse scaling
            outputs_np_flat = outputs_np.reshape(-1, outputs_np.shape[-1])
            labels_np_flat = labels_np.reshape(-1, labels_np.shape[-1])

            # Denormalizes
            outputs_denorm_flat = flow_scaler.inverse_transform(outputs_np_flat)
            labels_denorm_flat = flow_scaler.inverse_transform(labels_np_flat)

            # Reshape back to original shape
            outputs_denorm = outputs_denorm_flat.reshape(outputs_np.shape)
            labels_denorm = labels_denorm_flat.reshape(labels_np.shape)

            predictions.extend(outputs_denorm)
            actuals.extend(labels_denorm)

            # Calculate errors
            for t in range(pred_horizon):
                mse = np.mean((outputs_denorm[:, t] - labels_denorm[:, t]) ** 2)
                rmse = np.sqrt(mse)
                mae = np.mean(np.abs(outputs_denorm[:, t] - labels_denorm[:, t]))
                denominator = np.abs(outputs_denorm[:, t]) + np.abs(labels_denorm[:, t])
                smape = np.mean(2 * np.abs(outputs_denorm[:, t] - labels_denorm[:, t]) / (denominator + epsilon)) * 100
                smape = np.clip(smape, 0, 100)  # SMAPE can go up to 200%

                errors['mse'][t].append(mse)
                errors['rmse'][t].append(rmse)
                errors['mae'][t].append(mae)
                errors['mape'][t].append(smape)

    # Create a DataFrame to store the mean and standard deviation of errors
    error_stats = {
        'Hour': [],
        'Mean RMSE': [],
        'Std RMSE': [],
        'Mean MAE': [],
        'Std MAE': [],
        'Mean MAPE': [],
        'Std MAPE': []
    }

    # mean and standard deviation of errors for each hour
    for i in range(pred_horizon):
        mean_rmse = np.mean(errors['rmse'][i])
        std_rmse = np.std(errors['rmse'][i])
        mean_mae = np.mean(errors['mae'][i])
        std_mae = np.std(errors['mae'][i])
        mean_mape = np.mean(errors['mape'][i])
        std_mape = np.std(errors['mape'][i])

        error_stats['Hour'].append(i + 1)
        error_stats['Mean RMSE'].append(mean_rmse)
        error_stats['Std RMSE'].append(std_rmse)
        error_stats['Mean MAE'].append(mean_mae)
        error_stats['Std MAE'].append(std_mae)
        error_stats['Mean MAPE'].append(mean_mape)
        error_stats['Std MAPE'].append(std_mape)

    error_df = pd.DataFrame(error_stats)

    return np.array(predictions), np.array(actuals), error_df

# Evaluate the model
# predictions, actuals, error_df = evaluate_model_and_collect_data(model, test_loader, device, 'flow_scaler.gz', pred_horizon=n_steps_out)

# drive_path = '/content/drive/MyDrive/LSTM-ATT'

# predictions_filename = f'predictions_s{scenario}_12_{pred_hor}.gz'
# actuals_filename = f'actuals_s{scenario}_12_{pred_hor}.gz'

# joblib.dump(predictions, predictions_filename)
# joblib.dump(actuals, actuals_filename)

# !cp {predictions_filename} {drive_path}
# !cp {actuals_filename} {drive_path}

# Display the error statistics DataFrame
# error_df


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
predictions_path = '/content/drive/MyDrive/ATT_Bi_LSTM_RESULTS/predictions'
actuals_path = '/content/drive/MyDrive/ATT_Bi_LSTM_RESULTS/actuals'
error_path = '/content/drive/MyDrive/ATT_Bi_LSTM_RESULTS/errorMetrics'
final_results_path = '/content/drive/MyDrive/ATT_Bi_LSTM_RESULTS/final_results'

def save_to_drive(predictions, actuals, error_df, scenario, pred_hor):
    predictions_filename = f'predictions_s{scenario}_12_{pred_hor}.gz'
    actuals_filename = f'actuals_s{scenario}_12_{pred_hor}.gz'
    error_df_filename = f'errorMetrics_s{scenario}_12_{pred_hor}.csv'

    print('Saving predictions and actuals for scenario ', scenario, ' and prediction horizon ', pred_hor)
    error_df.to_csv(error_df_filename, index=False, sep = ';')
    joblib.dump(predictions, predictions_filename)
    joblib.dump(actuals, actuals_filename)

    !cp {predictions_filename} {predictions_path}
    !cp {actuals_filename} {actuals_path}
    !cp {error_df_filename} {error_path}

    # Remove the files from the local environment
    os.remove(predictions_filename)
    os.remove(actuals_filename)
    os.remove(error_df_filename)


    print('Done')
    print('\n')

In [ ]:
scenarios = [4,8]
prediction_horizons = [3,6,12]

batch_size = 32

# Create empty dataframe for times for training in current scenario and prediction horizon and best validation loss
times_df = pd.DataFrame(columns=['Scenario', 'Prediction Horizon', 'Time'])
best_losses_df = pd.DataFrame(columns=['Scenario', 'Prediction Horizon', 'Best Validation Loss'])

for scenario in scenarios:
  for prediction_horizon in prediction_horizons:
    # Choose full_data for each scenario sX
    inputs = full_data_list[scenario-1]
    labels = flow_data_scaled.copy()

    days_for_training = int(334*0.7)
    days_for_testing = int(334*0.2) + 1
    days_for_validation = int(334*0.1)

    test_steps = days_for_testing * 24
    validation_steps = days_for_validation * 24
    training_steps = days_for_training * 24

    inputs_train = inputs[:training_steps]
    inputs_validation = inputs[training_steps : training_steps + validation_steps]
    inputs_test = inputs[training_steps + validation_steps :]
    labels_train = labels[:training_steps]
    labels_validation = labels[training_steps : training_steps + validation_steps]
    labels_test = labels[training_steps + validation_steps :]

    n_steps_in, n_steps_out = 12, prediction_horizon
    X_train, y_train            = create_nodata_sequences(inputs_train,       labels_train,       n_steps_in, n_steps_out)
    X_validation, y_validation  = create_nodata_sequences(inputs_validation,  labels_validation,  n_steps_in, n_steps_out)
    X_test, y_test              = create_nodata_sequences(inputs_test,        labels_test,        n_steps_in, n_steps_out)


    X_train_tensor = torch.tensor(X_train).float()
    y_train_tensor = torch.tensor(y_train).float()
    X_test_tensor = torch.tensor(X_test).float()
    y_test_tensor = torch.tensor(y_test).float()
    X_validation_tensor = torch.tensor(X_validation).float()
    y_validation_tensor = torch.tensor(y_validation).float()

    train_data_final = TensorDataset(X_train_tensor, y_train_tensor)
    test_data_final = TensorDataset(X_test_tensor, y_test_tensor)
    validation_data_final = TensorDataset(X_validation_tensor, y_validation_tensor)

    batch_size = 32
    train_loader = DataLoader(train_data_final, shuffle=False, batch_size=batch_size)
    test_loader = DataLoader(test_data_final, shuffle=False, batch_size=batch_size)
    validation_loader = DataLoader(validation_data_final, shuffle=False, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    input_features, output_features = X_train_tensor.shape[2], y_train_tensor.shape[2]
    dropout_rate = 0.2
    model = ATT_BiLSTMModel(input_dim=input_features,
                      hidden_dim = 300,
                      num_layers = 3,
                      output_dim = output_features,
                      pred_horizon = prediction_horizon,
                      dropout = dropout_rate)
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print('Training for scenario ', scenario, ' and prediction horizon ', prediction_horizon)
    best_val_loss, time_to_train = train_model(model, train_loader, validation_loader, criterion, optimizer, num_epochs=100, patience=20)
    print(f"Best validation loss: {best_val_loss:.6f}")

    # Save validation loss in dataframe
    best_row = {'Scenario': scenario, 'Prediction Horizon': prediction_horizon, 'Best Validation Loss': best_val_loss}
    best_losses_df = pd.concat([best_losses_df, pd.DataFrame([best_row])], ignore_index=True)
    model.load_state_dict(torch.load('/content/best_model.pth'))

    # Save time it took to train in the times dataframe
    time_row = {'Scenario': scenario, 'Prediction Horizon': prediction_horizon, 'Time': time_to_train}
    times_df = pd.concat([times_df, pd.DataFrame([time_row])], ignore_index=True)

    print('Evaluating for scenario ', scenario, ' and prediction horizon ', prediction_horizon)
    predictions, actuals, error_df = evaluate_model_and_collect_data(model, test_loader, device, 'flow_scaler.gz', pred_horizon=n_steps_out)

    save_to_drive(predictions, actuals, error_df, scenario, prediction_horizon)

#Save the times and losses dataframes as csvs
times_df.to_csv('times_df_48.csv', index=False, sep = ';')
best_losses_df.to_csv('best_losses_df_48.csv', index=False, sep = ';')

!cp times_df_48.csv {final_results_path}
!cp best_losses_df_48.csv {final_results_path}

print('*********************************TESTS DONE!*********************************')

In [ ]:
scenarios = [1,2,3,5,6,7,9,10]
prediction_horizons = [3,6,12]

batch_size = 32

# Create empty dataframe for times for training in current scenario and prediction horizon and best validation loss
times_df = pd.DataFrame(columns=['Scenario', 'Prediction Horizon', 'Time'])
best_losses_df = pd.DataFrame(columns=['Scenario', 'Prediction Horizon', 'Best Validation Loss'])

for scenario in scenarios:
  for prediction_horizon in prediction_horizons:
    # Choose full_data for each scenario sX
    full_data = full_data_list[scenario-1]

    days_for_training = int(334*0.7)
    days_for_testing = int(334*0.2) + 1
    days_for_validation = int(334*0.1)

    test_steps = days_for_testing * 24
    validation_steps = days_for_validation * 24
    training_steps = days_for_training * 24

    training_data = full_data[:training_steps]
    validation_data = full_data[training_steps : training_steps + validation_steps]
    test_data = full_data[training_steps + validation_steps :]

    training_data_values = training_data.values
    test_data_values = test_data.values
    validation_data_values = validation_data.values

    n_features = full_data.shape[1]
    n_steps_in, n_steps_out = 12, prediction_horizon
    X_train, y_train = create_sequences(training_data_values, n_steps_in, n_steps_out, num_nodes)
    X_validation, y_validation = create_sequences(validation_data_values, n_steps_in, n_steps_out, num_nodes)
    X_test, y_test = create_sequences(test_data_values, n_steps_in, n_steps_out, num_nodes)

    X_train_tensor = torch.tensor(X_train).float()
    y_train_tensor = torch.tensor(y_train).float()
    X_test_tensor = torch.tensor(X_test).float()
    y_test_tensor = torch.tensor(y_test).float()
    X_validation_tensor = torch.tensor(X_validation).float()
    y_validation_tensor = torch.tensor(y_validation).float()

    train_data_final = TensorDataset(X_train_tensor, y_train_tensor)
    test_data_final = TensorDataset(X_test_tensor, y_test_tensor)
    validation_data_final = TensorDataset(X_validation_tensor, y_validation_tensor)

    batch_size = 32
    train_loader = DataLoader(train_data_final, shuffle=False, batch_size=batch_size)
    test_loader = DataLoader(test_data_final, shuffle=False, batch_size=batch_size)
    validation_loader = DataLoader(validation_data_final, shuffle=False, batch_size=batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    input_features, output_features = X_train_tensor.shape[2], y_train_tensor.shape[2]
    dropout_rate = 0.2
    model = ATT_BiLSTMModel(input_dim=input_features,
                      hidden_dim = 300,
                      num_layers = 3,
                      output_dim = output_features,
                      pred_horizon = prediction_horizon,
                      dropout = dropout_rate)
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print('Training for scenario ', scenario, ' and prediction horizon ', prediction_horizon)
    best_val_loss, time_to_train = train_model(model, train_loader, validation_loader, criterion, optimizer, num_epochs=100, patience=20)
    print(f"Best validation loss: {best_val_loss:.6f}")

    # Save validation loss in dataframe
    best_row = {'Scenario': scenario, 'Prediction Horizon': prediction_horizon, 'Best Validation Loss': best_val_loss}
    best_losses_df = pd.concat([best_losses_df, pd.DataFrame([best_row])], ignore_index=True)
    model.load_state_dict(torch.load('/content/best_model.pth'))

    # Save time it took to train in the times dataframe
    time_row = {'Scenario': scenario, 'Prediction Horizon': prediction_horizon, 'Time': time_to_train}
    times_df = pd.concat([times_df, pd.DataFrame([time_row])], ignore_index=True)

    print('Evaluating for scenario ', scenario, ' and prediction horizon ', prediction_horizon)
    predictions, actuals, error_df = evaluate_model_and_collect_data(model, test_loader, device, 'flow_scaler.gz', pred_horizon=n_steps_out)

    save_to_drive(predictions, actuals, error_df, scenario, prediction_horizon)

#Save the times and losses dataframes as csvs
times_df.to_csv('times_df.csv', index=False, sep = ';')
best_losses_df.to_csv('best_losses_df.csv', index=False, sep = ';')

!cp times_df.csv {final_results_path}
!cp best_losses_df.csv {final_results_path}

print('*********************************TESTS DONE!*********************************')

In [ ]:
#Make a loop that changes the separation in all the error metrics csv files to ';'
for scenario in scenarios:
  for prediction_horizon in prediction_horizons:
    df = pd.read_csv(f'errorMetrics_s{scenario}_12_{prediction_horizon}.csv')
    df.to_csv(f'errorMetrics_s{scenario}_12_{prediction_horizon}.csv', index=False, sep=';')

### Gathering and preparing data for plotting

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
import datetime

#Loop to plot the actuals vs the predictions of each scenario for the same pred_horizon in the same plot
predictions_s1 = joblib.load('predictions_s1_12_3.gz')
predictions_s2 = joblib.load('predictions_s2_12_3.gz')
predictions_s3 = joblib.load('predictions_s3_12_3.gz')
actuals = joblib.load('actuals_s1_12_3.gz')
# Plotting everythin in the same graph for a certain sensor and a certain range
sensor_id = 10
start_date = 25*24
end_date = 31*24
show_day_of_week = True  # Set to True to show day of the week instead of full date
sensor_predictions_s1 = predictions_s1[start_date:end_date, 0, sensor_id-1]
sensor_predictions_s2 = predictions_s2[start_date:end_date, 0, sensor_id-1]
sensor_predictions_s3 = predictions_s3[start_date:end_date, 0, sensor_id-1]
sensor_actuals = actuals[start_date:end_date, 0, sensor_id-1]
dates = [datetime.datetime(2023, 10, 19) + datetime.timedelta(hours=i) for i in range(start_date, end_date)]
plt.figure(figsize=(10, 5))
plt.plot(dates, sensor_predictions_s1, label='Predictions S1', marker='', linestyle='dashed')
plt.plot(dates, sensor_predictions_s2, label='Predictions S2', marker='', linestyle='dashdot')
plt.plot(dates, sensor_predictions_s3, label='Predictions S3', marker='', linestyle='dotted')
plt.plot(dates, sensor_actuals, label='Actual Values', marker='', linestyle='-')
plt.title(f"Predictions vs Actual Values for Sensor {sensor_id}")
plt.xlabel('Days')
plt.ylabel('Flow (Veh/h)')
plt.legend()
plt.grid(True)
if show_day_of_week:
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%A'))  # Day of the week
else:
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

    # Rotation for dates
plt.gca().xaxis.set_major_locator(mdates.DayLocator())
plt.gcf().autofmt_xdate()
plt.show()


In [ ]:


import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
import datetime

def plot_sensor_data(predictions, actuals, sensor_id, start_date, end_date, show_day_of_week=False):
    sensor_predictions = predictions[start_date:end_date, 0, sensor_id-1]
    sensor_actuals = actuals[start_date:end_date, 0, sensor_id-1]

    # Generate a range of dates
    dates = [datetime.datetime(2023, 10, 19) + datetime.timedelta(hours=i) for i in range(start_date, end_date)]

    plt.figure(figsize=(10, 5))

    plt.plot(dates, sensor_predictions, label='Predictions', marker='', linestyle='-')
    plt.plot(dates, sensor_actuals, label='Actual Values', marker='', linestyle='-')
    plt.title(f"Predictions vs Actual Values for Sensor {sensor_id}")
    plt.xlabel('Days')
    plt.ylabel('Flow (Veh/h)')
    plt.legend()
    plt.grid(True)

    if show_day_of_week:
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%A'))  # Day of the week
    else:
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))  # Full date

    plt.gca().xaxis.set_major_locator(mdates.DayLocator())
    plt.gcf().autofmt_xdate()  # Rotation for dates

    plt.show()

# Example usage:
sensor_id = 5
start_date = 25*24
end_date = 31*24
show_day_of_week = True  # Set to True to show day of the week instead of full date

plot_sensor_data(predictions, actuals, sensor_id, start_date, end_date, show_day_of_week)


## Hyperparameter optimization


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np


# Hyperparameters to test
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]


input_features = X_train_tensor.shape[2]
output_features = y_train_tensor.shape[2]
pred_horizon = 12
# Store results for plotting
results = []

# Setting the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


for dropout_rate in dropout_rates:
    model = ATT_BiLSTMModel(input_dim=input_features, hidden_dim=300, num_layers=3, output_dim=output_features, pred_horizon=pred_horizon, dropout=dropout_rate)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    print(f"Training with dropout_rate={dropout_rate}")
    best_val_loss, time_to_train  = train_model(model, train_loader, validation_loader, criterion, optimizer, num_epochs=100, patience=20)
    print(f"Best validation loss: {best_val_loss:.6f}")
    results.append({
        'dropout_rate': dropout_rate,
        'best_val_loss': best_val_loss,
        'time_to_train': time_to_train
    })

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)
results_df

In [ ]:
results_df.to_csv('hyperparameters_v2.csv', index=False, sep = ';')
!cp hyperparameters_v2.csv /content/drive/MyDrive/results

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np


# Hyperparameters to test
hidden_dims = [100, 200, 300, 400]
num_layers = [1, 2, 3, 4]
batch_sizes = [32, 64, 128, 256]
learning_rates = [0.0001, 0.0005, 0.001]

input_features = X_train_tensor.shape[2]
output_features = y_train_tensor.shape[2]
pred_horizon = 12
# Store results for plotting
results = []

# Setting the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training loop
for batch_size in batch_sizes:
    train_loader = DataLoader(train_data_final, shuffle=False, batch_size=batch_size)
    validation_loader = DataLoader(validation_data_final, shuffle=False, batch_size=batch_size)

    for hidden_dim in hidden_dims:
        for num_layer in num_layers:
            for lr in learning_rates:
                model = LSTMModel(input_dim=input_features, hidden_dim=hidden_dim, num_layers=num_layer, output_dim=output_features, pred_horizon=pred_horizon)
                model.to(device)
                optimizer = optim.Adam(model.parameters(), lr=lr)
                criterion = nn.MSELoss()

                print(f"Training with batch_size={batch_size}, hidden_dim={hidden_dim}, num_layers={num_layer}, learning_rate={lr}")
                best_val_loss, time_to_train  = train_model(model, train_loader, validation_loader, criterion, optimizer, num_epochs=100, patience=20)

                results.append({
                    'batch_size': batch_size,
                    'hidden_dim': hidden_dim,
                    'num_layers': num_layer,
                    'learning_rate': lr,
                    'best_val_loss': best_val_loss,
                    'time_to_train': time_to_train
                })

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)
results_df

In [ ]:
results_df.to_csv('hyperparameters_v2.csv', index=False, sep = ';')
!cp hyperparameters_v2.csv /content/drive/MyDrive/results

## Device information

In [ ]:
# Check CPU information
print("CPU Information:")
!lscpu

# Check GPU information
print("\nGPU Information:")
!nvidia-smi

# Check RAM information
print("\nRAM Information:")
!free -h

# Check disk space
print("\nDisk Space Information:")
!df -h

In [ ]:
import numpy as np

# Check for NaNs or Infs in actuals and predictions
print("NaNs in actuals:", np.isnan(actuals).any())
print("NaNs in predictions:", np.isnan(predictions).any())
print("Infs in actuals:", np.isinf(actuals).any())
print("Infs in predictions:", np.isinf(predictions).any())

# Check for minimum values in actuals and predictions
print("Min value in actuals:", np.min(actuals))
print("Min value in predictions:", np.min(predictions))
print("Max value in actuals:", np.max(actuals))
print("Max value in predictions:", np.max(predictions))

# Check the shapes of actuals and predictions
print("Shape of actuals:", actuals.shape)
print("Shape of predictions:", predictions.shape)

print("Number of zeros in actuals:", (actuals == 0).sum())
print("Number of zeros in predictions:", (predictions == 0).sum())
print("Percentage of zeros in actuals", (actuals == 0).sum() / actuals.size * 100)
print("Percentage of zeros in predictions", (predictions == 0).sum() / predictions.size * 100)

# Calculate MAPE with added epsilon to avoid division by zero
epsilon = 1e-10
mape = np.abs((actuals - predictions) / (actuals + epsilon)) * 100

print("percentage of MAPE over 100", (mape > 100).sum() / mape.size * 100)

# Check for any unexpected negative values
print("Minimum MAPE value:", mape.min())
print("Maximum MAPE value:", mape.max())
print("Sample MAPE values:", mape.flatten()[:10])

# Debugging the problematic MAPE calculation
# Find the specific instances where MAPE is negative
problematic_indices = np.where((np.abs((actuals - predictions) / (actuals + epsilon)) * 100) < 0)
print("Problematic indices:", problematic_indices)

# Examine the values at problematic indices
print("Actuals at problematic indices:", actuals[problematic_indices])
print("Predictions at problematic indices:", predictions[problematic_indices])


In [ ]:
zeros_count_total = (training_data == 0).sum().sum()
print("Total number of zeros in the DataFrame:", zeros_count_total)
columns_with_zeros = (training_data == 0).any().sum()

print("Number of columns with at least one zero:", columns_with_zeros)

mask = (test_data == 0).any()

columns_with_zeros = training_data.columns[mask]

print("Columns containing all one zero:", list(columns_with_zeros))